# Store Sales - Time Series Forecasting

### Contexto:
Los pronósticos no solo se limitan a la meteorología. Los gobiernos pronostican crecimiento económico. Los científicos intentan predecir la población futura. Y las empresas pronostican la demanda de productos, es una tarea común de los científicos de datos profesionales. Los pronósticos son especialmente relevantes para las tiendas de abarrotes tradicionales, que deben escoger delicadamente la cantidad de inventario que se debe comprar.
Para predecir un mayor precisión que productos deben comprar y en cuantas cantidades. Adivinar los artículos populares que se agotan rápidamente no es muy eficiente, esto puede generar una pérdida de ingresos y clientes molestos. Gracias al aprendizaje automático, se puede ayudar a garantizar a que los  clientes pueda tener suficientes productos correctos en el momento correcto.

Los métodos actuales de pronóstico subjetivo para el comercio minorista tienen pocos datos que los respalden y es poco probable que se automaticen. El problema se vuelve aún más complejo a medida que los minoristas agregan nuevas ubicaciones con necesidades únicas, nuevos productos, gustos estacionales en constante transición y marketing de productos impredecible.

### Meta:
Con lo anterior en mente debemos construir un modelo que prediga con mayor precisión las ventas unitarias de miles de artículos vendidos en diferentes tiendas Favorita.
Practicará sus habilidades de aprendizaje automático con un conjunto de datos de capacitación accesible de fechas, tiendas e información de artículos, promociones y ventas de unidades.

<br>

<div align="center"><img width="500" height="500" src='https://upload.wikimedia.org/wikipedia/commons/0/0f/Corporaci%C3%B3n_Favorita_Logo.png'></div>

<br>

### Equipo Cortesanas de IA:
- __Leonardo Alvarado__
- __Oscar Delgadillo__
- __David Guzmán__
- __Enrique Santos__



In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
#Import other dependencies

# Visualization libraries
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

#pandas dataframe configurations
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format

### Obtenemos la ubicación de nuestros datos

In [10]:
DATA_PATH = '../input/store-sales-time-series-forecasting/'

In [11]:
for f in os.listdir(DATA_PATH):
    print(f)

### Creamos diferentes DataFrames para cada una de las tablas que se nos presentan

In [12]:
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

df_stores = pd.read_csv(os.path.join(DATA_PATH, 'stores.csv'))
df_transactions = pd.read_csv(os.path.join(DATA_PATH, 'transactions.csv')).sort_values(['store_nbr', 'date'])
df_oil = pd.read_csv(os.path.join(DATA_PATH, 'oil.csv'))
df_holidays = pd.read_csv(os.path.join(DATA_PATH, 'holidays_events.csv'))

### Juntamos información que consideramos relevante para un train y un test más completos

In [13]:
train = pd.merge(df_train,df_stores, on='store_nbr',how='left')
test = pd.merge(df_test,df_stores, on='store_nbr',how='left')

train = pd.merge(train,df_oil, on='date',how='left')
test = pd.merge(test,df_oil, on='date',how='left')
    
holidays_events = df_holidays.set_index('date').sort_index()

train = pd.merge(train.set_index('date'),holidays_events,on='date',how='left')
train = train.rename(columns={"type_x":"type","type_y":"holidays_type"})

test = pd.merge(test.set_index('date'),holidays_events,on='date',how='left')
test = test.rename(columns={"type_x":"type","type_y":"holidays_type"})

In [14]:
df_train.head()

In [15]:
df_train.sales

In [16]:
# Datetime
df_train["date"] = pd.to_datetime(df_train['date'])
df_test["date"] = pd.to_datetime(df_test.date)
df_transactions["date"] = pd.to_datetime(df_transactions.date)

In [17]:
# Data types
df_train.onpromotion = df_train.onpromotion.astype("float16")
df_train.sales = df_train.sales.astype("float32")
df_stores.cluster = df_stores.cluster.astype("int8")

df_train.head()

In [18]:
df_train.describe()

### EDA (Exploratory Data Analysis) Muy Básico

In [19]:
average_sales = df_train.groupby('date').mean()['sales']
average_sales.head()

In [20]:
average_sales.plot(style='.', figsize=(20,10));

En el gráfico de dispersión de arriba podemos observar el promedio de ventas que hay por día durante desde 2013 hasta el 2018 aproximadamente.

In [21]:
avg_train = average_sales.copy()

In [22]:
moving_avg = avg_train.rolling(window=365, center=True, min_periods=183).mean()
avg_train.plot(style='.', figsize=(20,10));
moving_avg.plot();

En el gráfico de arriba podemos observar como se va comportando los el promedio de ventas en el tiempo.

### EDA Básico con transactions

In [23]:
df_transactions.head()

In [24]:
average_transactions = df_transactions.groupby('date').mean()['transactions']
average_transactions.head()

In [25]:
average_transactions.plot(style='.', figsize=(20,10));

En este gráfico de dispersión podemos ver el comportamiento del promedio de transacciones diarias durante el tiempo.

In [26]:
avg_transactions = average_transactions.copy()

moving_avg = avg_transactions.rolling(window=365, center=True, min_periods=183).mean()
avg_transactions.plot(style='.', figsize=(20,10));
moving_avg.plot();

Por otra parte en este gráfico se añade el promedio de transacciones anuales en el tiempo.

### Se realizó el análisis y limpieza de datos de las familias de productos vendidas en las tiendas Favorita, localizadas en Ecuador

Para este análisis primero observamos que tiendas han vendido mayor cantidad de productos.

In [27]:
df_train_temp = df_train.groupby(['date', 'store_nbr']).sales.sum().reset_index()
df_train_temp

Generamos una tabla auxiliar que junta diferentes valores para observar las ventas y transacción que hay en las diferentes tiendas con respecto al tiempo.

In [28]:
df_aux_merged = pd.merge(df_train_temp, df_transactions, how='left')
df_aux_merged.columns

Obtenemos la correlación que tienen las ventas con las transacciones.

In [29]:
pearson_corr = df_aux_merged.corr('pearson')['sales'].loc['transactions']
spearman_corr = df_aux_merged.corr('spearman')['sales'].loc['transactions']
print(f'pearson corr = {pearson_corr:.4f} and spearman corr = {spearman_corr:.4f}')

In [30]:
df_sorted_trans = df_transactions.sort_values(['store_nbr', 'date'])

px.line(df_sorted_trans, x='date', y='transactions', color='store_nbr', title='Transactions by date and store')

En el gráfico de arriba el número de transacciones que se realiza por tienda, dandonos cuenta que la número 44 es la que tiene el mayor número de transacciones y que en general tiene un pico durante festividades navideñas (a finales de todos lo años).

In [31]:
aux_a = df_transactions.copy()
aux_a['year'] = aux_a.date.dt.year
aux_a['month'] = aux_a['date'].dt.month

aux_a.head()

In [32]:
px.box(aux_a, x='year', y='transactions', color='month', title='Box plot transactions')

En el gráfico de cajas de arriba nos podemos dar cuenta que el mayor número de transacciones se da en diciembre (por las festividades), mientras que los otros meses los valores no tienden a variar mucho.

In [33]:
aux_a = df_transactions.set_index('date').resample('M').transactions.mean().reset_index()
aux_a['year'] = aux_a.date.dt.year

px.line(aux_a, x='date', y='transactions', color='year', title='Monthly transactions')

En esta gráfica de arriba tambien podemos ver las transacciones por año teniendo mayor cantidad de transacciones en el 2014 y una mayor pendiente negativa en 2015 derivando en un minimo número de transacciones durante el año 2016.

### EDA Básico con stores

In [34]:
fam_sales = df_train.groupby(['family']).sales.sum()
fam_sales.head()

In [35]:
px.line(fam_sales)

En esta gráfica desplegamos las ventas totales por tipo de producto en toda la corporación.

In [36]:
df_train_temp = df_train.groupby(['date', 'store_nbr']).sales.sum().reset_index()
df_train_temp

In [37]:
df_train_temp2 = df_train.groupby(['date', 'family']).sales.sum().reset_index()
df_train_temp2

In [39]:
px.line(df_train_temp2, x = 'date', y = 'sales', color = 'family')

Con un mayor análisis nos pudimos dar cuenta con el gráfico de arriba que la mayor familia que se vende en la tienda son del tipo GROCERY I (árticulos de primera necesidad), además al investigar un poco sobre el máximo que tenemos, nos pudimos dar cuenta que es debido a un terremoto el 16 de abril de 2016, que podría ser una buena decisión quitar esos datos para que no afecte nuestro modelo de predicción, ya que no podemos saber cuando va a ocurrir un terremoto.

In [40]:
#df_stores.head()
df_stores_group = df_stores.copy()
df_stores_group = df_stores_group.groupby(['city']).count().reset_index()
#df_stores_group.head()
px.bar(df_stores_group, x='city', y='type', title='Stores by city')

En el histograma de arriba mostramos la cantidad de tiendas que hay por ciudad.

In [41]:
df_stores_state = df_stores_group.groupby(['state']).count().reset_index()
#df_stores_group.head()
px.bar(df_stores_state, x='state', y='type', title='Stores by state')

En el histograma de arriba mostramos la cantidad de tiendas que hay por estado.

In [45]:
train.isnull().sum()

In [46]:
train["dcoilwtico"] = train["dcoilwtico"].fillna(method='bfill',axis=0)
train["dcoilwtico"].plot(figsize=(20, 10))

En el gráfico de arriba se aprecia una gran decaída del petróleo por el tiempo, que teóricamente afectaría la salud económica, y como pudimos ver en tablas de trasacciones la caida del precio del petroleo en 2015-2016 también afecto el número de transacción que se dio esos años.

In [47]:
df_train_temp2 = df_train.groupby(['date','store_nbr']).sales.sum().reset_index()
df_train_temp2

df_train_stores = pd.merge(df_train_temp2, df_stores, how = 'left')

df_train_stores.columns

px.line(df_train_stores, x = 'date', y = 'sales', color = 'state', title = 'Sales by state')

Con la información de los histogramas de arriba graficamos las ventas por estado, donde podemos ver que el estado que tiene una mayor cantidad de ventas es en Pichincha, esto puede darse porque se encuentra la capital kito y porque tiene la mayor cantidad de tiendas.